In [1]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -v -p numpy,sklearn,pandas
%watermark -v -p cv2,PIL,matplotlib
%watermark -v -p torch,torchvision,torchaudio,pytorch_lightning
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False


from IPython.display import display, HTML
display(HTML('<style>.container { width:%d%% !important; }</style>' % 90))

CPython 3.6.9
IPython 7.16.1

numpy 1.19.4
sklearn 0.24.0
pandas 1.1.5
CPython 3.6.9
IPython 7.16.1

cv2 4.5.1
PIL 6.2.2
matplotlib 3.3.3
CPython 3.6.9
IPython 7.16.1

torch 1.8.0.dev20210103+cu101
torchvision 0.9.0.dev20210103+cu101
torchaudio not installed
pytorch_lightning 1.2.0


## Install Depends Libraries

In [9]:
!apt install ffmpeg
!pip3 install youtube_dl
!pip3 install moviepy --trusted-host pypi.tuna.tsinghua.edu.cn  --index-url https://pypi.tuna.tsinghua.edu.cn/simple

You should consider upgrading via the '/usr/bin/python3.6 -m pip install --upgrade pip' command.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
You should consider upgrading via the '/usr/bin/python3.6 -m pip install --upgrade pip' command.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 65 not upgraded.


In [68]:
import os, io, time, math, base64
import numpy as np
import pandas as pd
import tempfile
import youtube_dl
import subprocess

import torch
import cv2
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

## Data Process

In [4]:
YOUTUBE_PREFIX = 'https://www.youtube.com/watch?v='
DATASET_PREFIX = '/data/datasets/cv/countix'

REP_OUT_TIME = 0.5 # for (repetition_start - REP_OUT_TIME, repetition_end + REP_OUT_TIME)
SOCKS5_PROXY = 'socks5://127.0.0.1:1881'

### Countix Dataset Download and Crop

```
    vs           cs                ce            ve
    |             |0.5           0.5|             |
    |-------|---------|---------|------|----------|
            |         |         |      |            
            ks       rs        re      ke
    
vs: the video start
ve: the video end
ks: the kinetics start
ke: the kinetics end
rs: repetition start
re: repetition end
cs: clip video start
ce: clip video end 

```

In [66]:
YDL_OPTS = {
    'format': 'mp4',
    'proxy': SOCKS5_PROXY,
    'quiet': False,
    'max_filesize': 30000000, # 30MB
}

def video_download_crop(vid, ss, to, raw_dir, out_dir):
    raw_file = f'{raw_dir}/{vid}.mp4'
    out_file = f'{out_dir}/{vid}_{ss}_{to}.mp4'

    if os.path.exists(out_file):
        return out_file

    if not os.path.exists(raw_file):
        YDL_OPTS['outtmpl'] = raw_file
        with youtube_dl.YoutubeDL(YDL_OPTS) as ydl:
            ydl.download([f'{YOUTUBE_PREFIX}{vid}'])

    if os.path.exists(raw_file):
        fps = MODEL_Z_FRAMES / (to - ss)
        cmd = 'ffmpeg -i %s -s 112x112 -ss %s -to %s -r %s -an %s' % (
                raw_file, ss, to, fps, out_file)
        subprocess.call(cmd, shell=True)
        return out_file

    return None

def data_preprocess(data_prefix, phase):
    df = pd.read_csv(f'{data_prefix}/countix_{phase}.csv')
    raw_dir = f'{data_prefix}/raw/{phase}'
    out_dir = f'{data_prefix}/{phase}'
    os.makedirs(raw_dir, exist_ok=True)
    os.makedirs(out_dir, exist_ok=True)
    df['file_name'] = ''
    for idx, row in df.iterrows():
        if phase = 'test':
            vid, ks, ke, rs, re, count, file_name = row
        else:
            vid, _, ks, ke, rs, re, count, file_name = row
            
        cs = max([ks, rs - REP_OUT_TIME])
        ce = min([ke, re + REP_OUT_TIME])
        try:
            out_file = video_download_crop(vid, cs, ce, raw_dir, out_dir)
            if out_file is not None:
                df.loc[idx, 'file_name'] = out_file
            else:
                print('download or crop [%s] fail' % vid)
        except Exception as err:
            print('%s' % err)
    sub_df = df[df['file_name'].notnull()]
    sub_df.to_csv(f'{data_prefix}/sub_countix_{phase}.csv', index=False, header=True)
    return sub_df

In [65]:
# data_preprocess(DATASET_PREFIX, 'test')
# data_preprocess(DATASET_PREFIX, 'val')
# data_preprocess(DATASET_PREFIX, 'train')

In [ ]:
# ZzqBD58txYE,skiing slalom,21,31,21.000000,31.000000,4

In [109]:
%%bash

rm -rf /data/tmp1.mp4
ffmpeg -i /data/datasets/cv/countix/ZzqBD58txYE.mp4 -v 0 -s 112x112 -ss 10 -to 31 -frames 64 -an /data/tmp1.mp4 

In [110]:
frames = []
cap = cv2.VideoCapture('/data/tmp1.mp4')
while cap.isOpened():
    ret, frame = cap.read()
    if ret is False:
        break
    frames.append(frame)
cap.release()

len(frames)

64

In [94]:
64 / (3 - 1)

32.0

### Countix Dataset Sample Display

In [81]:
df = pd.read_csv('/data/datasets/cv/countix/sub_countix_test.csv')

In [83]:
sub_df

,video_id,kinetics_start,kinetics_end,repetition_start,repetition_end,count,file_name
0,0CnQ-_e5Sl8,17,27,18.500000,20.233333,4,countix/test/0CnQ-_e5Sl8_18.0_20.733333.mp4
1,0CnQ-_e5Sl8,17,27,21.633333,24.166667,6,countix/test/0CnQ-_e5Sl8_21.133333_24.666667.mp4
2,0dZohHw4j_U,10,20,16.500000,20.000000,8,countix/test/0dZohHw4j_U_16.0_20.mp4
3,0rvWeDPPQ3M,7,17,7.000000,17.000000,3,countix/test/0rvWeDPPQ3M_7_17.mp4
4,1KKLVWl9FPE,2,12,9.800000,12.000000,4,countix/test/1KKLVWl9FPE_9.3_12.mp4
...,...,...,...,...,...,...,...
2635,7C5DuIuCsOY,37,47,37.000000,47.000000,7,countix/test/7C5DuIuCsOY_37_47.mp4
2636,7Uv-26v3k00,21,31,21.000000,30.984871,5,countix/test/7Uv-26v3k00_21_31.mp4
2637,7xGq3iNmiB4,0,10,0.000000,9.994318,10,countix/test/7xGq3iNmiB4_0_10.mp4
2638,8NXFPGqHLdE,55,65,55.000000,58.882770,2,countix/test/8NXFPGqHLdE_55_59.382769999999994...


In [80]:
sample1 = '/data/datasets/cv/countix/test/0CnQ-_e5Sl8_18.0_20.733333.mp4'

cap = cv2.VideoCapture(sample1)

df

,video_id,kinetics_start,kinetics_end,repetition_start,repetition_end,count,file_name
0,0CnQ-_e5Sl8,17,27,18.500000,20.233333,4,countix/test/0CnQ-_e5Sl8_18.0_20.733333.mp4
1,0CnQ-_e5Sl8,17,27,21.633333,24.166667,6,countix/test/0CnQ-_e5Sl8_21.133333_24.666667.mp4
2,0dZohHw4j_U,10,20,16.500000,20.000000,8,countix/test/0dZohHw4j_U_16.0_20.mp4
3,0rvWeDPPQ3M,7,17,7.000000,17.000000,3,countix/test/0rvWeDPPQ3M_7_17.mp4
4,1KKLVWl9FPE,2,12,9.800000,12.000000,4,countix/test/1KKLVWl9FPE_9.3_12.mp4
...,...,...,...,...,...,...,...
2637,7xGq3iNmiB4,0,10,0.000000,9.994318,10,countix/test/7xGq3iNmiB4_0_10.mp4
2638,8NXFPGqHLdE,55,65,55.000000,58.882770,2,countix/test/8NXFPGqHLdE_55_59.382769999999994...
2639,9qdhLeQaP1k,205,215,206.166667,215.000000,5,NaN
2640,9-EjQ50hEg0,118,128,118.000000,125.640974,3,NaN


In [79]:
for idx, row in df.iterrows():
    frames = []
    cap = cv2.VideoCapture(f'/data/datasets/cv/{row.file_name}')
    while cap.isOpened():
        ret, frame = cap.read()
        if ret is False:
            break
        frames.append(frame)
    cap.release()
    print(len(frames), row.file_name)

65 countix/test/0CnQ-_e5Sl8_18.0_20.733333.mp4
65 countix/test/0CnQ-_e5Sl8_21.133333_24.666667.mp4
64 countix/test/0dZohHw4j_U_16.0_20.mp4
64 countix/test/0rvWeDPPQ3M_7_17.mp4
64 countix/test/1KKLVWl9FPE_9.3_12.mp4
64 countix/test/1pcMlkxdxzs_13_23.mp4
65 countix/test/1q_O0aQGIDc_111_114.20833300000001.mp4
64 countix/test/1v8uiFp8SMY_6.667835_8.735569.mp4
64 countix/test/1v8uiFp8SMY_9.570737_11.471638.mp4
64 countix/test/1v8uiFp8SMY_10.538372_12.506006.mp4
49 countix/test/1v8uiFp8SMY_13.841675_15.775943.mp4
64 countix/test/11PyjUjkXh4_17_27.mp4
64 countix/test/159j6-j6okU_5_11.572739.mp4
64 countix/test/159j6-j6okU_10.606106_14.208708999999999.mp4
65 countix/test/198vC1u8dUQ_0_3.9048910000000006.mp4
64 countix/test/2Pg5MZReFQQ_4.470637_11.mp4
64 countix/test/2ThmlbRQQtY_10_17.433332999999998.mp4
65 countix/test/2ThmlbRQQtY_16.466667_20.mp4
65 countix/test/2U42k7qiDnw_19.566667000000002_25.mp4
65 countix/test/2VK9_8vPQeg_28.67284_33.mp4
64 countix/test/2gkOOA90ZEM_69_72.56973599999999.m

64 countix/test/4EK46v4E_Gs_58_62.9.mp4
64 countix/test/4U7RwpSr9uw_49.5_54.63333299999999.mp4
64 countix/test/4U7RwpSr9uw_55.966667_59.mp4
65 countix/test/5BzNU_RTfsI_14.134967999999999_21.674842.mp4
64 countix/test/5DHECfA7B2g_6.576923_9.807692.mp4
0 nan
64 countix/test/5IIYJEVp6-8_13_14.7.mp4
65 countix/test/5cHzjKLrWvs_13_22.075242000000003.mp4
64 countix/test/5k9OjJ63VAE_44.835669_46.628128000000004.mp4
64 countix/test/5yF9hfc7OOQ_13_14.766667000000002.mp4
64 countix/test/5yF9hfc7OOQ_16.733333_18.7.mp4
64 countix/test/59S58PFjGEo_0_10.mp4
64 countix/test/6GwjXluAztY_2_12.mp4
64 countix/test/6Rdq_umMifo_16.5_21.583333.mp4
64 countix/test/6b2gujvZ8_g_7.1_15.mp4
64 countix/test/6dLztarU4KI_72_74.43333299999999.mp4
64 countix/test/6dLztarU4KI_75.233333_81.466667.mp4
59 countix/test/6hhzX5eATaQ_31.866667_33.966667.mp4
64 countix/test/6j3SrttFr3Y_2.666667_6.9.mp4
65 countix/test/624rDx0cR_Y_55.040246999999994_57.mp4
64 countix/test/68DInYWQVyM_24.253235999999998_26.028586999999998.mp4
6

64 countix/test/29DiOrHgpDo_4.978261_12.mp4
64 countix/test/3YmziQwoOf4_10_14.125.mp4
64 countix/test/3YmziQwoOf4_15.6875_20.mp4
64 countix/test/3gN6B1TVGo0_0_10.mp4
65 countix/test/4XY3q2XDWVQ_10_16.533333.mp4
64 countix/test/4mcMlXLLMLg_4.366667_7.0.mp4
65 countix/test/4oWwmqosjUQ_104_111.10660700000001.mp4
64 countix/test/5B4RNxhnn14_1.7000000000000002_10.mp4
64 countix/test/5rvfyevveJk_34_38.833333.mp4
64 countix/test/6LGRDUviogw_27_34.82.mp4
65 countix/test/66f0xCdzqmU_3.1036040000000003_10.mp4
0 nan
64 countix/test/7IAUOfCOCOk_7_10.38.mp4
64 countix/test/8DI-SXuUlzk_0_7.540374000000001.mp4
64 countix/test/8Y589apIlzY_43.2_48.033333.mp4
64 countix/test/8g-bUiKAE4E_45.733333_50.9.mp4
64 countix/test/8rgXXT_3-4Y_85.166667_90.666667.mp4
64 countix/test/-LUwbfPn1rk_34.28_42.mp4
64 countix/test/-mko88bEoDo_2_7.505005000000001.mp4
64 countix/test/-qdz0akV2Vg_131_136.5.mp4
64 countix/test/0M9-yrl6jM8_114.05_119.21666699999999.mp4
64 countix/test/0M9-yrl6jM8_118.25_122.95.mp4
64 countix/t

64 countix/test/4jg0CpLCK4U_11_19.066667000000002.mp4
64 countix/test/4mP11TbmkXE_6_15.8.mp4
0 nan
64 countix/test/5S5YI-ZnXdc_7_10.43627.mp4
64 countix/test/6EVgBrzpqr0_11.174341_14.mp4
64 countix/test/7QhkXjqPudw_51.413651_56.106660999999995.mp4
65 countix/test/7aGLtmILhQ0_29_33.637471000000005.mp4
65 countix/test/8HHe2p23jgA_7_12.266666.mp4
65 countix/test/8bcEcA9vqyk_25_31.583334000000004.mp4
64 countix/test/9XRY2teNZJg_29.100601_36.707207000000004.mp4
64 countix/test/-UpxqvdpoqE_21.566667000000002_25.333333.mp4
64 countix/test/-0HwkO7TRmc_34.638472_40.mp4
64 countix/test/-0aB_FGVkfU_24_31.533333000000002.mp4
64 countix/test/AdcBCnF7LtM_2.1_8.54.mp4
64 countix/test/A7__d4ALkGc_4.966667_10.366667.mp4
0 nan
64 countix/test/C4lGwH3cSGw_26_32.038872.mp4
65 countix/test/D4G5WGOUqMM_147.2_150.966667.mp4
0 nan
64 countix/test/ESFyQ2-85iI_10.233333_13.5.mp4
64 countix/test/FJinw46tHHQ_13_15.333332999999998.mp4
65 countix/test/FfB_XXeD1Do_4_9.466667.mp4
65 countix/test/Fp62gCq6eHE_43_46.069

65 countix/test/24jMzIDjKV4_18.333333_21.7.mp4
64 countix/test/4eDmQ9qsHH8_94_104.mp4
65 countix/test/41JEU6v7SCk_217_221.07023700000002.mp4
65 countix/test/5I-ZgKZPDq8_13.2_15.566667.mp4
64 countix/test/5xnwoOpuYjE_20_30.mp4
64 countix/test/6CsTKSWemKk_3.766667_7.6666669999999995.mp4
65 countix/test/6vRMMj8jJDQ_0.36666699999999997_10.mp4
64 countix/test/7Wknr-ZEHSY_4_14.mp4
64 countix/test/8BVHOkapiBg_64.809065_73.mp4
0 nan
65 countix/test/9ulkxMUx-A8_14.516667000000002_17.0.mp4
0 nan
0 nan
64 countix/test/AjuEcPcll1c_10_19.514723999999998.mp4
64 countix/test/AnPbAwriUcs_78.953209_82.mp4
0 nan
64 countix/test/BAQpT-dc7gg_15.466667000000001_23.mp4
0 nan
0 nan
64 countix/test/BoHqfUH3uZY_3_13.mp4
64 countix/test/B8Ehw3DYvTs_26_36.mp4
64 countix/test/B9ROFytAMlQ_68_71.645753.mp4
64 countix/test/CfSsolTnP64_4_14.mp4
64 countix/test/CzT4NmKEFAM_84_86.33516800000001.mp4
65 countix/test/CzT4NmKEFAM_86.519686_90.42259.mp4
64 countix/test/CzT4NmKEFAM_90.707207_92.77494200000001.mp4
64 countix/

...

64 countix/test/0cg1ExC7iHc_4_14.mp4
64 countix/test/0dKWtUHxdDk_37_47.mp4
64 countix/test/0i4pJszQREs_26.95_34.933334.mp4
64 countix/test/0jbQEarO6mA_3.7800000000000002_10.mp4
65 countix/test/0kmupN9LQ60_61.470636999999996_68.mp4
60 countix/test/0oPK_19z5bo_0_5.66.mp4
64 countix/test/01TOcMcVR10_10_20.mp4
64 countix/test/032RVBemMmU_6_16.mp4
64 countix/test/0-OXF3re1xA_1_11.mp4
64 countix/test/1RhYjuLxxZc_0_10.mp4
64 countix/test/1V29uSrMkD4_25_35.mp4
63 countix/test/1Y95Ek8EPBA_10_19.609109.mp4
62 countix/test/1aPKv2xzLUo_0_9.175342.mp4
64 countix/test/1e3eUsZ-TOY_5_15.mp4
60 countix/test/1fVl3uLObNA_0_5.771939.mp4
64 countix/test/1kJ0mowT9Ao_0_10.mp4
63 countix/test/1mhaIVipCIg_1_10.577282.mp4
64 countix/test/1ne3aBqopfQ_5_15.mp4
64 countix/test/1nzu76mGvNg_47_57.mp4
63 countix/test/1owAFas5QDE_12_21.509009.mp4
64 countix/test/1pwKN5Yiayo_18_28.mp4
65 countix/test/1ypugybsQSQ_15_22.006507.mp4
65 countix/test/2G8lONku7A0_6_15.242075.mp4
63 countix/test/2VxjQUM-Noo_19_28.9427760000000

65 countix/test/2hqz8jKesJA_7.583333999999999_10.216667.mp4
51 countix/test/2hqz8jKesJA_13.766667000000002_15.933333.mp4
64 countix/test/3EE-3w-jSw0_6.566667_9.666667.mp4
64 countix/test/3KCG23FwiQI_83.82_86.42.mp4
64 countix/test/3KCG23FwiQI_86.7_88.48.mp4
64 countix/test/3KwXiItcdiQ_3.533334_5.7.mp4
64 countix/test/3Yfq-6VmBjo_0.6200000000000001_6.34.mp4
64 countix/test/3eN1qxiMTdc_5.29361_7.41983.mp4
64 countix/test/3eN1qxiMTdc_8.255151_10.256236.mp4
64 countix/test/3jVOrJwZH9g_25.233333_28.166666999999997.mp4
64 countix/test/3jVOrJwZH9g_29.75_32.966667.mp4
64 countix/test/3l1ECSaMiZA_12_15.9.mp4
64 countix/test/3oxwM_2Nxhk_16.666667_19.866667.mp4
64 countix/test/3oxwM_2Nxhk_21.8_26.mp4
65 countix/test/3x52JWOsQ4A_9.287431_12.534316.mp4
64 countix/test/4ALh5-PfkAQ_16_26.mp4
64 countix/test/4Eswv164XOU_17_18.883333.mp4
65 countix/test/4Eswv164XOU_22.033333_26.266667.mp4
64 countix/test/4Kpdgc1FVi8_24_27.75.mp4
64 countix/test/4Kpdgc1FVi8_30.958334000000004_33.166666.mp4
64 countix/te

64 countix/test/-ugkPBCizgE_49.58_58.mp4
64 countix/test/AFCHK4xs6ec_50.133333_53.mp4
64 countix/test/AHRD-zBovXw_5_15.mp4
64 countix/test/AaOlLV5exA0_11_13.19016.mp4
64 countix/test/A5W9nS_uAZs_55.733332999999995_58.733332999999995.mp4
65 countix/test/A6ZRET5K-zQ_1.7999999999999998_10.mp4
64 countix/test/A6oI0-EnazY_99_102.875.mp4
64 countix/test/A-OE6GUo0vg_0_10.mp4
65 countix/test/BGW_2xjQA1A_43.155172_46.mp4
64 countix/test/BZdNy4jMbn0_0_10.mp4
64 countix/test/BdcukeUOxNI_34_36.4.mp4
0 nan
0 nan
64 countix/test/DD68HHOkMHw_15_17.02.mp4
0 nan
65 countix/test/aVF_agu1Wf0_14_18.733333.mp4
64 countix/test/chMmUN-F-5c_11_19.133333.mp4
64 countix/test/ciDqk1DeMns_23.533333_26.mp4
65 countix/test/c-jfDzarxyI_131.415455_138.mp4
64 countix/test/dAAYKrYiggA_5.005506_10.mp4
64 countix/test/dEHZNV1zXVs_19.555556_23.mp4
64 countix/test/0P33tknjbO8_37.166667_44.566666999999995.mp4
64 countix/test/0UCLSiC3yGI_11.2_14.666667000000002.mp4
61 countix/test/08yJ9OPFGN4_54.337171_61.mp4
64 countix/test

65 countix/test/9CKSEoFso1I_86.633333_90.mp4
64 countix/test/9ibfE4m4vvs_125_135.mp4
0 nan
0 nan
64 countix/test/ApgPpGcwYBU_1_11.mp4
0 nan
64 countix/test/A7bOLpYpxc8_76.966667_86.mp4
64 countix/test/BMCFFYvDzdY_29_39.mp4
64 countix/test/BsazicGxz1c_4_14.mp4
65 countix/test/BxMYwvCfiv0_25_29.433333.mp4
64 countix/test/DNO3PS2YjZ8_14_24.mp4
64 countix/test/FP9pT-7g9Bg_28_38.mp4
64 countix/test/Haju0fSoxgI_18_28.mp4
63 countix/test/KwifkOWFG_o_1_10.9.mp4
64 countix/test/M8p6xxlbAv8_8_18.mp4
65 countix/test/NIXydUXw0Fc_20.433332999999998_29.mp4
64 countix/test/NI7rtkzAV3Y_11_17.706205999999998.mp4
64 countix/test/N_NeYFIZri4_16_20.795963.mp4
64 countix/test/OXneyCJwcx0_1_11.mp4
65 countix/test/OgrzXCpx2NI_51.133967_60.65916.mp4
64 countix/test/QFg_W7LKKk4_11.981558_20.mp4
64 countix/test/QMCxptKi1W0_67_73.42.mp4
65 countix/test/QT60QBt-dFE_27_35.508008000000004.mp4
63 countix/test/QbjNlUVqqgc_50_59.666667000000004.mp4
64 countix/test/Qs0u33I31eI_39_49.mp4
64 countix/test/RwRR-MF5D0w_31_4

64 countix/test/1D9faa72mQM_66_76.mp4
64 countix/test/1LLWWT4UnKs_23_33.mp4
65 countix/test/1VSrIj3FEKQ_2_5.766667.mp4
64 countix/test/1VSrIj3FEKQ_8.766667_12.mp4
64 countix/test/1emv8ASvZvs_11_21.mp4
0 nan
0 nan
64 countix/test/22MSSoe2FlY_21_31.mp4
64 countix/test/3BbTEQ8xHM4_15_25.mp4
65 countix/test/34bG4TiXIF4_86_93.633333.mp4
64 countix/test/39crfK5qYG4_12_22.mp4
64 countix/test/4N8sMHnPWOo_12_22.mp4
65 countix/test/4XRIy6eWjSQ_6.805305000000001_12.mp4
64 countix/test/4-j_u9WuwMM_15.0_17.mp4
64 countix/test/5XdCxNmWEpM_183_185.716667.mp4
64 countix/test/5cR-NDQWw-g_4_14.mp4
64 countix/test/6thvzmwGd4k_23_33.mp4
65 countix/test/67GmGx8FON0_8.933333_15.mp4
65 countix/test/-PA0ixGEHFc_3.0333330000000003_10.mp4
64 countix/test/-Vw4q6p68Mw_7_17.mp4
64 countix/test/-cKHpHzs9N0_22_30.733334000000003.mp4
65 countix/test/-cKHpHzs9N0_30.516666999999998_32.mp4
65 countix/test/-zcB7azu7fo_255_257.166667.mp4
65 countix/test/-0bSj1Z8Vj8_16_24.833333.mp4
65 countix/test/1G_5EuHPLE8_6.8333330000

In [74]:
len(frames)

65

## References

1. https://arxiv.org/pdf/2006.15418.pdf

2. https://colab.research.google.com/github/google-research/google-research/blob/master/repnet/repnet_colab.ipynb#scrollTo=76L5XFonl_Bw